# Ensemble Learning and Random Forests

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

##  Imports

In [83]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.datasets import make_moons
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier


## Models

### Voting technique

In [106]:
X, y = make_moons(100, noise=0.15)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [118]:
svc_clf, rf_clf, log_clf = SVC(probability=Tr), RandomForestClassifier(), LogisticRegression()

voting_clf = VotingClassifier(
    estimators=[('svc', svc_clf), ('rf', rf_clf), ('log',log_clf)],
    voting='hard', 
)
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('svc',
                              SVC(C=1.0, break_ties=False, cache_size=200,
                                  class_weight=None, coef0=0.0,
                                  decision_function_shape='ovr', degree=3,
                                  gamma='scale', kernel='rbf', max_iter=-1,
                                  probability=True, random_state=None,
                                  shrinking=True, tol=0.001, verbose=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gin...
                                                     random_state=None,
                                                     verbose=0,
                                                     warm_start=False)),

In [119]:
for clf in (svc_clf, rf_clf, log_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__,  accuracy_score(y_test, y_pred))

SVC 1.0
RandomForestClassifier 1.0
LogisticRegression 0.9
VotingClassifier 0.95


###  Bagging technique